# Importing Libs

In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import transforms, datasets
import torchvision

from torch.utils.tensorboard import SummaryWriter
import comet_ml
from comet_ml import Experiment

In [2]:
torch.__version__, torch.cuda.is_available()

('1.13.0+cu116', True)

In [3]:
# Comet Initialisation 
#comet_ml.init(project_name="DCGAN")

In [4]:
experiment = comet_ml.Experiment(
    api_key="w8mmQk2vAHgMuQwj1xR1tRHns",
    project_name="DCGAN"
)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch, tensorboard. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/rohitpawar2406/dcgan/55f2774d575b425dbde39a4698c5a769



In [5]:
# Parameters
# these will all get logged
params = {
    "batch_size": 32,
    "epochs": 4,
    "optimizer": 'adam',
    "lr" : 0.0002,
    "image_size" : 64,
    "channels_noise" : 256,
    "featuresDiscriminator" : 16,  # original paper 64 but for MNIST its 16 okay for me!!
    "featureGenerator" : 16         # original paper 64 but for MNIST its 16 okay for me!!
}
device = "cuda" if torch.cuda.is_available() else "cpu"
experiment.log_parameters(params)


In [6]:
#experiment.log_code(file_name="DCGAN.ipynb")

# Class Discriminator

In [7]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features) -> None:
        super(Discriminator, self).__init__();
        # Input data -> N x channels_size x 64 x 64 
        self.net = nn.Sequential(
            nn.Conv2d(channels_img, features, kernel_size=4, stride=2, padding=1), 
            nn.LeakyReLU(0.2),
            # output -> N x features x 32 x 32

            nn.Conv2d(features, features*2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(0.2),
            # output -> N x features*2 x 16 x 16

            nn.Conv2d(features*2, features*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(0.2),
            # output -> N x features*4 x 8 x 8 

            nn.Conv2d(features*4, features*8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2),
            # output -> N x features*8 x 4 x 4

            nn.Conv2d(features*8, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
         )
    
    def forward(self, x):
        return self.net(x)

# Class Generator

In [8]:
class Generator(nn.Module):
    def __init__(self, channels_noise:int, channels_img:int, features:int) -> None:
        super(Generator, self).__init__();

        self.gen = nn.Sequential(
            # Input -> N x channel_noise x 1 x 1

            nn.ConvTranspose2d(channels_noise, features*16, kernel_size=4, stride=1, padding=0), # Output -> N x features*16 x 4 x 4
            nn.BatchNorm2d(features*16),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(features*16, features*8, kernel_size=4, stride=2, padding=1), # Output -> N x features*8 x 8 x 8
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(features*8, features*4, kernel_size=4, stride=2, padding=1), # Output -> N x features*4 x 16 x 16
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(features*4, features*2, kernel_size=4, stride=2, padding=1), # Output -> N x features*2 x 32 x 32
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(features*2, channels_img, kernel_size=4, stride=2, padding=1), # Output -> N x channels_img (3 or 1) x 32 x 32
            nn.Tanh()
    )
    
    def forward(self, x):
        return self.gen(x)

In [9]:
# j = Generator(100, 1 , 64)
# j1 = j(torch.randn(32,100,1,1))
# j1.shape

In [10]:
# Transforms
transforms = transforms.Compose([
    transforms.Resize(params['image_size']),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

# Dataset and DataLoader Class

In [11]:
dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms, download=False)
dataloader = DataLoader(dataset, batch_size=params['batch_size'], shuffle=True)

In [12]:
# for i,(j,k) in enumerate(dataloader):
#     print(j.shape)

# Object of Class Generator and Discriminator

In [13]:
disc = Discriminator(1, params['featuresDiscriminator']).to(device)
gen = Generator(params["channels_noise"], 1, params['featureGenerator']).to(device)

# Setting Up Optimisers and Loss Function

In [14]:
optmDisc = optim.Adam(disc.parameters(), lr=params['lr'], betas=(0.5,0.999))
optmGen = optim.Adam(gen.parameters(), lr=params['lr'], betas=(0.5,0.999))

# Loss Functions
criterion = nn.BCELoss()

real_label = 1 
fake_label = 0

fixed_noise = torch.randn(64, params['channels_noise'], 1,1).to(device)

# Training Loop

In [15]:
print("Started training!!!")
for epoch in range(params['epochs']):
    for batch_indx, (data, target) in enumerate(dataloader):
        data = data.to(device)
        batch_size = data.shape[0]

        ## Train Discriminator: max log(D(x)) + log(1-D(G(z)))
        disc.zero_grad()
        # 1) Counting for log(D(x)) for this loss(data, labels) 
        disc_real = disc(data).reshape(-1)
        labels_real  = (torch.ones(batch_size)*0.9).to(device)
        loss_real = criterion(disc_real,labels_real)
        D_x = disc_real.mean().item()

        # 2) Counting for max of log(1-D(G(z)))
        noise = torch.randn(batch_size, params['channels_noise'], 1, 1).to(device)
        genFake = gen(noise)    # output -> batch_size x 3 x 64 x 64
        label_fake = (torch.ones(batch_size)*0.1).to(device)

        discFake = disc(genFake.detach()).reshape(-1)    # output -> batch_size x channels x 1 x 1 and after reshape [multiply all of them value]
        loss_fake = criterion(discFake, label_fake)

        lossD = loss_real + loss_fake 
        lossD.backward()
        optmDisc.step()

        ## Train Generator: min log(D(G(z)))
        gen.zero_grad()
        labelsGenFake = torch.ones(batch_size).to(device)
        output = disc(genFake).reshape(-1)
        lossGen = criterion(output, labelsGenFake)
        lossGen.backward()
        optmGen.step()

        with torch.no_grad():
            fake = gen(fixed_noise)
            imgGridReal = torchvision.utils.make_grid(data[:32], normalize=True)
            imgGridFake = torchvision.utils.make_grid(fake[:32], normalize=True)
            
        # if batch_indx % 100 == 0:
        #     print(f'Epoch[{epoch}]')

    print("Epoch Num: ", epoch)




Started training!!!


KeyboardInterrupt: 